## ● Web crawler-Information of Flickr user's Hometown&Residence MingHsun 2016/01/06
### 網路爬蟲是啥?
* 網路爬蟲（web crawler），也叫網路蜘蛛（spider），是一種用來自動瀏覽全球資訊網的網路機器人。其目的一般為編纂網路索引。網路搜尋引擎等站點通過爬蟲軟體更新自身的網站內容或其對其他網站的索引。網路爬蟲可以將自己所存取的頁面儲存下來，以便搜尋引擎事後生成索引供用戶搜尋。爬蟲存取網站的過程會消耗目標系統資源。不少網路系統並不默許爬蟲工作。因此在存取大量頁面時，爬蟲需要考慮到規劃、負載，還需要講「禮貌」，或者就是[不被發現]。
* 網路爬蟲主要有兩個部分:
>1.獲取資訊：在給定一個URL時，我們可以使用HTTP協定的方法去對於伺服器送出Request請求，並且取得Response回應，在我們爬蟲一般的情況就是對於伺服器送出GET Request來取得HTML檔案的Response。<br>
2.處理與分析內容：擁有網頁的內容之後，我們就可以透過正規表示、字串處理、split等的各種技巧來過濾抓到我們所要的資料。

Reference:https://en.wikipedia.org/wiki/Web_crawler
## Python主要模組操作與介紹:<br>

###  ● urlib2
> urllib.request 是一個用來從URL取得資料的Python模組。它提供了urlopen 這個函數，非常簡單的介面能但他可以接受多種不同的協議。其他關於authentication、cooki等也提供handler或opener等物件操作。以下我們將使用urlopen來取得網站資訊。

###  ● BeautifulSoup
>   BeautifulSoup是甚麼呢? 簡單來說，它用來parsing html的內容，擷取你想要的HTML tag以及內容資料。網頁就是由一堆HTML tag 階層性的組合而成，BeautifulSoup的功能

## 實作-網路爬蟲 Flickr使用者的家鄉、居住地
* 我預先已經有一些使用者的ID,Flickr並沒有提供或取使用者居住地的API,但是在使用者的個人資訊網頁上，一般會留家鄉、居住地的資訊，因而我們透過網路爬蟲的方式或取這些資料。

In [17]:
import csv
uid_file=open("UserID.csv","r")
# uid_file=open("try_uid.csv","r")
data_dict={}
for row in csv.reader(uid_file):
#     print row[1]
    if row[1] not in data_dict:
        data_dict.setdefault(row[1],[])
uid_file.close()

In [19]:
print len(data_dict) #總計有幾筆id

32762


## Crawling

* 1.使用urllib獲取資料
* 2.用re套件，正規表示式Regular Expression ，去搜尋家鄉和目前居住地此tag的位置
* 3.用Beautifulsoup來parsing資料

In [10]:
import re
import urllib2
from bs4 import BeautifulSoup
for uid in data_dict:
    try:
        response = urllib2.urlopen('https://www.flickr.com/people/'+str(uid)+"/")
        html = response.read()
        soup = BeautifulSoup(html, "html.parser")
        profile_section = soup.find('div', {'id': "a-bit-more-about"}) 
        soup2=BeautifulSoup(str(profile_section), "html.parser")
        match_h=re.search('家鄉', str(profile_section))
        if match_h:
            home=soup2.find('dt',text='家鄉︰').findNext('dd').contents
#             print home[0]
            data_dict[uid].append(home[0])
        else:
            data_dict[uid].append("NoData")
        match_r=re.search('目前居住地', str(profile_section))
        if match_r:
            residence=soup2.find('dt',text='目前居住地：').findNext('dd').span.span.contents
#             print residence[0]
            data_dict[uid].append(residence[0])
        else:
            data_dict[uid].append("NoData")
    except:
        continue

In [11]:
print data_dict

{'100028363@N02': ['NoData', 'NoData'], '10000800@N03': ['NoData', 'NoData'], '100004432@N04': ['NoData', 'NoData'], '100030370@N05': ['NoData', 'NoData'], '100004885@N05': ['NoData', 'NoData'], '100011529@N05': ['NoData', 'NoData'], '100013766@N03': ['NoData', 'NoData'], '100015994@N08': ['NoData', 'NoData'], '100018075@N02': ['NoData', 'NoData'], '100009241@N07': [u'\u9cf3\u5c71', u'\u6797\u5712, \u53f0\u7063']}


* 儲存爬蟲結果

In [16]:
csv_list=[["uid","home","residence"]]
for uid in data_dict:
    csv_list.append([str(uid),data_dict[uid][0].encode("utf8"),data_dict[uid][1].encode("utf8")])
home_f=open("home_match.csv","w")
w = csv.writer(home_f)  
w.writerows(csv_list)  
home_f.close()